In [1]:
#installing faiss for semantic search https://github.com/facebookresearch/faiss
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

--2023-03-03 19:59:48--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=600&X-Amz-Date=20230303T195949Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEBsaCXVzLWVhc3QtMSJHMEUCIGGITwobxtJNYhQPILhNWIQ8Hji3ja6FK5PvK1yltLTOAiEA3ve%2FYv4ZWvocOGMX%2FH4WxUsFvXzTnPQYYOOeZh%2FfQ4EqvAUIw%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw0NTU4NjQwOT

In [2]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.7 MB/s eta 0:00:00


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.4 MB/s eta 0:00:00


##Import packages

In [4]:
#Ignore Warning
import warnings
warnings.filterwarnings("ignore") 
# set up notebook to display multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from tqdm.notebook import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil
from collections import defaultdict
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate,Conv1D,MaxPool1D,Dropout

## Mount Google Drive to Colab environment

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
%cd /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/
!pwd
#!ls
print('Working Directory')
print(os.getcwd())
work_dir = "/content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/"

/content/gdrive/.shortcut-targets-by-id/1wHLGXlHrKWfJBcdQSt8SRTchDZIbjUV8/NW-MSDS_CapstoneProject
/content/gdrive/.shortcut-targets-by-id/1wHLGXlHrKWfJBcdQSt8SRTchDZIbjUV8/NW-MSDS_CapstoneProject
Working Directory
/content/gdrive/.shortcut-targets-by-id/1wHLGXlHrKWfJBcdQSt8SRTchDZIbjUV8/NW-MSDS_CapstoneProject


###Final Prediction

In [7]:
import re
#preprocessing questions and answers.
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

In [8]:

def preprocess(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


In [9]:
from transformers import AutoTokenizer, TFAutoModel
def return_biobert_tokenizer_model():
    '''returns pretrained biobert tokenizer and question extractor model'''
    biobert_tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")
    question_extractor_model=tf.keras.models.load_model(work_dir + 'question_extractor_model')
    return biobert_tokenizer,question_extractor_model

In [10]:
def return_gpt2_tokenizer_model():
    '''returns pretrained gpt2 tokenizer and gpt2 model'''
    gpt2_tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
    tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained(work_dir + "gpt2_model/medical_gpt2_model_new_2_104_90000")
    return gpt2_tokenizer,tf_gpt2_model

In [13]:
train_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/train_gpt_data.pkl')
question_bert = train_gpt_QnA["Q_FFNN_embeds"].tolist()
answer_bert = train_gpt_QnA["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])

question_index = faiss.IndexFlatIP(question_bert.shape[-1])
answer_index.add(answer_bert)
question_index.add(question_bert)

In [14]:
#defining function to prepare the data for gpt inference
#https://github.com/ash3n/DocProduct
def preparing_gpt_inference_data(question,question_embedding,gpt2_tokenizer):
  topk=20
  scores,indices=answer_index.search(
                  question_embedding.astype('float32'), topk)
  q_sub=train_gpt_QnA.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: ' % (
                        question)
  encoded_len=len(gpt2_tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(gpt2_tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return gpt2_tokenizer.encode(line)[-1024:]

In [33]:
#function to generate answer given a question and the required answer length
import random
def predict_answer(question,answer_len=100):
  preprocessed_question=preprocess(question)
  question_len=len(preprocessed_question.split(' '))
  truncated_question=preprocessed_question
  biobert_tokenizer,question_extractor_model= return_biobert_tokenizer_model()
  gpt2_tokenizer,tf_gpt2_model= return_gpt2_tokenizer_model()
  if question_len>500:
    truncated_question=' '.join(preprocessed_question.split(' ')[:500])
  encoded_question= biobert_tokenizer.encode(truncated_question)
  max_length=512
  padded_question=tf.keras.preprocessing.sequence.pad_sequences(
      [encoded_question], maxlen=max_length, padding='post')
  question_mask=[[1 if token!=0 else 0 for token in question] for question in padded_question]
  embeddings=question_extractor_model({'question':np.array(padded_question),'question_mask':np.array(question_mask)})
  gpt_input=preparing_gpt_inference_data(truncated_question,embeddings.numpy(),gpt2_tokenizer)
  mask_start = len(gpt_input) - list(gpt_input[::-1]).index(4600) + 1
  input=gpt_input[:mask_start+1]
  if len(input)>(1024-answer_len):
   input=input[-(1024-answer_len):]
  output_sequences = tf_gpt2_model.generate(
            input_ids =tf.constant([np.array(input)]),
            max_length = 1024,
            # temperature = 0.7,
            num_beams=3, 
            no_repeat_ngram_size=2, 
            #num_return_sequences=1, 
            early_stopping=True,
            # top_k = 0,
            # top_p = 0.9,
            # repetition_penalty = 1,
            # do_sample = True,
            num_return_sequences = 3,
            # do_sample=True, 
            # top_k=0,
            pad_token_id=gpt2_tokenizer.eos_token_id
        )
  #gpt2_output=gpt2_tokenizer.decode(tf_gpt2_model.generate(input_ids=tf.constant([np.array(input)]),max_length=1024,temperature=0.7,pad_token_id=gpt2_tokenizer.eos_token_id)[0])
  gpt2_output=gpt2_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
  #print(gpt2_output.find('`ANSWER : '))
  if gpt2_output.find('`ANSWER : ') != -1:
    answer=gpt2_output.rindex('`ANSWER : ')
    ansIndex = answer+len('`ANSWER : ')
  elif gpt2_output.find('`ANSWER') != -1:
    answer=gpt2_output.rindex('`ANSWER')
    ansIndex = answer+len('`ANSWER')
  else:
    answer=gpt2_output.rindex('`ANSWER: ')
    ansIndex = answer+len('`ANSWER: ')

  return gpt2_output[ansIndex:]
  #return gpt2_output,
  

In [22]:
#function to generate answer given a question and the required answer length
import random
def predict_answer(question,answer_len=50):
  preprocessed_question=preprocess(question)
  question_len=len(preprocessed_question.split(' '))
  truncated_question=preprocessed_question
  biobert_tokenizer,question_extractor_model= return_biobert_tokenizer_model()
  gpt2_tokenizer,tf_gpt2_model= return_gpt2_tokenizer_model()
  if question_len>500:
    truncated_question=' '.join(preprocessed_question.split(' ')[:500])
  encoded_question= biobert_tokenizer.encode(truncated_question)
  max_length=512
  padded_question=tf.keras.preprocessing.sequence.pad_sequences(
      [encoded_question], maxlen=max_length, padding='post')
  question_mask=[[1 if token!=0 else 0 for token in question] for question in padded_question]
  embeddings=question_extractor_model({'question':np.array(padded_question),'question_mask':np.array(question_mask)})
  gpt_input=preparing_gpt_inference_data(truncated_question,embeddings.numpy(),gpt2_tokenizer)
  mask_start = len(gpt_input) - list(gpt_input[::-1]).index(4600) + 1
  input=gpt_input[:mask_start+1]
  if len(input)>(1024-answer_len):
   input=input[-(1024-answer_len):]
  output_sequences = tf_gpt2_model.generate(
            input_ids =tf.constant([np.array(input)]),
            max_length = 1024,
            top_k = 20, 
            top_p = 0.9,
            #temperature = 0.7,
            # num_beams=3, 
            #no_repeat_ngram_size=2, 
            # num_return_sequences=1, 
            early_stopping=True,
            # top_k = 0,
            # top_p = 0.9,
            repetition_penalty = 1,
            #do_sample = True,
            num_return_sequences = 1,
            # do_sample=True, 
            # top_k=0,
            pad_token_id=gpt2_tokenizer.eos_token_id
        )
  #gpt2_output=gpt2_tokenizer.decode(tf_gpt2_model.generate(input_ids=tf.constant([np.array(input)]),max_length=1024,temperature=0.7,pad_token_id=gpt2_tokenizer.eos_token_id)[0])
  #gpt2_output=gpt2_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
  #print(gpt2_output.find('`ANSWER : '))
  print("Output:\n" + 100 * '-')
  for i, sample_output in enumerate(output_sequences):
    gpt2_output = gpt2_tokenizer.decode(sample_output, skip_special_tokens = True)
    answer=gpt2_output.rindex('`ANSWER: ')
    print("{}: {}...".format(i, gpt2_output[answer:]))
    print('')
  #return gpt2_output,
  

In [34]:
answer=predict_answer('what are the signs of heart attack?')
print("answer:",answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1173 > 1024). Running this sequence through the model will result in indexing errors


answer: heart attacks are often caused by a combination of several factors including a viral infection or an underlying heart disease the most common cause of death is the inability of blood vessels to move in and out of your heart vessels in addition to being unable to breathe or move your arms or legs it can also be a result of an enlarged heart or a heart defect in one or both of those vessels there is no way to diagnose or treat the disease in any way other than to see your medical provider for an examination and


In [53]:
answer=predict_answer('How can I tell if I am having a heart attack?')
print("answer:",answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1104 > 1024). Running this sequence through the model will result in indexing errors


answer: hi if you are having heart attacks i would suggest that you see your medical provider for an exam to see if there is anything to do with your heart disease if so then you can get a blood test done to rule out any other possible causes of your problem so you will be properly diagnosed and treated for your disease good luck ps hope this helps your case hope you find a place for you to live hope it helps you well ps do hope your doctor is right thanks for reading this and if it works


In [35]:
answer=predict_answer('How to treat osteoarthritis?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1386 > 1024). Running this sequence through the model will result in indexing errors


hi if you have any questions about your medical history you can consult a gynecologist online   > <link> > > www <aadvocacy org > webmd medical reference from healthwise this information is not intended to replace the advice of a doctor health wise disclaims any liability for any injury or damage resulting from the ideas or products mentioned or the accompanying information © 1995 2011 healthcarewise for every health decision you make and for all its products and services including our services you agree to


In [36]:
answer=predict_answer('How to stop migraine attack?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


if you are having any symptoms of migraine then you should see your medical provider for an exam to rule out any other possible causes of your symptoms then go to your doctor and ask for a blood test to make sure you have no other side effects if so then it would be best for you to consult your health care provider and get tested for any side effect if possible then get checked out by your pharmacist and be properly diagnosed and treated for the symptoms you may be able to find a doctor in your


In [37]:
answer=predict_answer('what should i do if my baby is vomiting?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1165 > 1024). Running this sequence through the model will result in indexing errors


; if he or she starts to vomit it may be best to see your medical provider for a blood test to determine if there is any blood clotting in his or her lungs or intestines if you have any questions about this please call your health care provider and ask for blood tests to be done to rule out any possible blood clots in the blood vessels in your lungs   > <link> > >  <a href="http://www.healthwise.com">Healthwise</a


In [38]:
answer=predict_answer('why my eyes hurts sometimes?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1089 > 1024). Running this sequence through the model will result in indexing errors


; i know it sounds like you are having an allergic reaction to something that is going on in your eyelid but it would be best for you to go see your doctor and see what he or she can tell you about it if you have any other allergies you may be able to talk to him or her and get your hair back in order to help you get back to your normal hair growth and hair color you will have to wait a month or two before you go to the salon to find the right


In [39]:
answer=predict_answer('what should i do to avoid backaches')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1049 > 1024). Running this sequence through the model will result in indexing errors


if you have backache then you may want to see your medical provider for an exam to rule out any other possible causes of back aches then it may be best for you to consult your primary care provider and see if there is anything else that could be causing your symptoms if possible consult a gynecologist online   > <link> to open and read <a href="http://www.healthwise.com">a webmd medical reference from healthwise this information is not intended to


In [40]:
answer=predict_answer('How to prevent stomach flu?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1163 > 1024). Running this sequence through the model will result in indexing errors


if your doctor or health care professional knows your medical history and your history you should be able to talk with him or her about your concerns and concerns you can talk about it in person or online   > <link> > >  <a href="https://www.facebook.com/groups/medicinefda">groups</a> to help you find the right person for you to speak with and discuss your issues with your healthcare professional <br>  ><link type="


In [41]:
answer=predict_answer('Is conjunctivitis contagious?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1079 > 1024). Running this sequence through the model will result in indexing errors


no it does not seem to be contagious to people who are not infected by the virus but it can spread to other people if they are exposed to it they may become infected with other viruses or bacteria that are spread by direct contact or by contaminated needles or needles it may be possible for people to spread the infection to others if the infected person has a fever or cough or sneezing or if he or she has an enlarged lymph node or lung cancer the person who has the disease may also be exposed


In [42]:
answer=predict_answer('how to diagnose cancer?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 1024). Running this sequence through the model will result in indexing errors


if you have any questions about your diagnosis or treatment plan please call your primary care provider or health care professional   > <link> > >> <a href="https://www.healthwiseinc.com">www webmd medical reference from healthwise this information is not intended to replace the advice of a doctor health wise disclaims any liability for the decisions you make based on this content including your doctor's recommendations physician recommendations may refer you to an internist for further information consult a registered


In [43]:
answer=predict_answer('what are the alzheimer disease treatment')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1214 > 1024). Running this sequence through the model will result in indexing errors


the most common treatment for the disease is the use of anticonvulsants such as diclofenac and dicyclomine   a combination of these medications can be helpful in treating the symptoms of this disease and in preventing the spread of disease to other parts of your body including the brain and spinal cord the medication can also help reduce the risk of developing other diseases like heart disease or lung disease the medications are usually prescribed by your doctor or pharmacist to treat your symptoms


###Evaluation Metrics  
Bleu Score

In [45]:
from nltk.translate.bleu_score import sentence_bleu

In [51]:
def calc_bleu_score(question,answer):
  answer_len=100
  generated_answer=predict_answer(question,answer_len)
  reference = [answer.split(' ')]
  candidate = generated_answer.split(' ')
  #print(generated_answer)
  score = sentence_bleu(reference, candidate)
  return score

In [52]:
metric=calc_bleu_score('why my eyes hurts sometimes?','you are having an allergic reaction to something')
print(metric)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1089 > 1024). Running this sequence through the model will result in indexing errors


0.0656676176790682
